Install and Import Dependencies


In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

Load Model

In [2]:
interpreter = tf.lite.Interpreter(model_path='../lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [3]:
plt.imshow(tf.cast(np.squeeze(img), dtype=tf.int32))

NameError: name 'img' is not defined

In [4]:
img = frame.copy()

NameError: name 'frame' is not defined

In [5]:
img.shape

NameError: name 'img' is not defined

In [11]:
np.expand_dims(img, axis=0).shape

(1, 1, 192, 192, 3)

In [13]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 312,
  'shape': array([ 1,  1, 17,  3], dtype=int32),
  'shape_signature': array([ 1,  1, 17,  3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

Make Detections

In [19]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)

    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)

    # Rendering
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break

# cap.release()
# cv2.destroyAllWindows()

[[[[0.3816342  0.47243863 0.7739444 ]
   [0.3076588  0.5340491  0.75635135]
   [0.30373564 0.41676986 0.6918182 ]
   [0.32280326 0.58838725 0.76120937]
   [0.31686392 0.35054573 0.6067573 ]
   [0.4836949  0.66351175 0.76271737]
   [0.53041744 0.20586173 0.90792865]
   [0.77813095 0.7932118  0.42156383]
   [0.9727119  0.04896697 0.13364494]
   [0.7836683  0.7979231  0.03968555]
   [0.7690511  0.4191651  0.00870445]
   [0.86426806 0.6044674  0.01117906]
   [0.9676455  0.28834882 0.00748691]
   [0.7642487  0.73785806 0.00791857]
   [0.52141577 0.19621989 0.0360074 ]
   [0.7917242  0.7999701  0.00438854]
   [0.78297734 0.01011732 0.01014367]]]]
[[[[0.38322866 0.46997678 0.81046355]
   [0.3104145  0.53229475 0.6806372 ]
   [0.30407944 0.4149325  0.59930503]
   [0.32837266 0.58692515 0.7174207 ]
   [0.31749403 0.34972835 0.6439351 ]
   [0.48242682 0.659459   0.71770966]
   [0.527488   0.20589113 0.8737472 ]
   [0.7794597  0.79249823 0.38240504]
   [0.9705509  0.05040408 0.08874574]
   [0.786

KeyboardInterrupt: 

In [7]:
right_eye = keypoints_with_scores[0][0][2]
left_elbow = keypoints_with_scores[0][0][7]

In [8]:
shaped = np.squeeze(np.multiply(interpreter.get_tensor(interpreter.get_output_details()[0]['index']), [480,640,1])).astype(int)

In [9]:
for kp in shaped:
    ky, kx, kp_conf = kp
    print(int(ky), int(kx), kp_conf)


232 337 0
229 328 0
228 338 0
229 308 0
228 333 0
251 287 0
250 327 0
290 278 0
290 332 0
313 280 0
316 352 0
313 283 0
314 308 0
361 295 0
363 313 0
376 295 0
376 313 0


Draw Keypoints

In [10]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 


Draw Edges

In [14]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [15]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)